In [7]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from server.lib.BackTest import BackTest
from server.lib.models import BaseModel
from server.prebuilt.portfolioAllocations import PercentagePortfolioAllocation
from server.prebuilt.riskManagement import TakeProfitStopLoss
from server.custom.equitiesSelection import SpacsSelectionModel
from server.custom.strategies import SpacSignals

class SpacInsiderLogisticRegressionModel(BaseModel):
    def __init__(self):
        super().__init__()
        self.start_date = "2022-08-07"
        self.end_date = "2024-08-07"
        self.model_name = 'SPAC Insider Logistic Regression with Stop Loss 20 Model'
        self.equity_selection_management = SpacsSelectionModel(40, 5, 'Short', self.start_date, self.end_date)
        self.portfolio_management = PercentagePortfolioAllocation(1000000, 10)
        self.alpha_model = SpacSignals()
        self.risk_control = TakeProfitStopLoss(100, 20)

Test = BackTest(SpacInsiderLogisticRegressionModel)
Test.run()

[*********************100%***********************]  70 of 70 completed


0.0063381887394174915
0.003763542423047297


In [8]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from server.lib.BackTest import BackTest
from server.lib.models import BaseModel
from server.prebuilt.portfolioAllocations import PercentagePortfolioAllocation
from server.prebuilt.riskManagement import TakeProfitStopLoss
from server.custom.equitiesSelection import SpacsSelectionModel
from server.custom.strategies import SpacSignals

class SpacInsiderLogisticRegressionModel(BaseModel):
    def __init__(self):
        super().__init__()
        self.start_date = "2022-08-07"
        self.end_date = "2024-08-07"
        self.model_name = 'SPAC Insider Logistic Regression with Stop Loss 10 Model'
        self.equity_selection_management = SpacsSelectionModel(40, 5, 'Short', self.start_date, self.end_date)
        self.portfolio_management = PercentagePortfolioAllocation(1000000, 10)
        self.alpha_model = SpacSignals()
        self.risk_control = TakeProfitStopLoss(100, 10)

Test = BackTest(SpacInsiderLogisticRegressionModel)
Test.run()

[*                      3%                       ]  2 of 70 completed

[*********************100%***********************]  70 of 70 completed


0.006239052025253428
0.003755509631031672


In [9]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from server.lib.BackTest import BackTest
from server.lib.models import BaseModel
from server.prebuilt.portfolioAllocations import PercentagePortfolioAllocation
from server.prebuilt.riskManagement import TraillingStopLoss
from server.custom.equitiesSelection import SpacsSelectionModel
from server.custom.strategies import SpacSignals

class SpacInsiderLogisticRegressionModel(BaseModel):
    def __init__(self):
        super().__init__()
        self.start_date = "2022-08-07"
        self.end_date = "2024-08-07"
        self.model_name = 'SPAC Insider Logistic Regression with Trailling Stop Loss 20 Model'
        self.equity_selection_management = SpacsSelectionModel(40, 5, 'Short', self.start_date, self.end_date)
        self.portfolio_management = PercentagePortfolioAllocation(1000000, 10)
        self.alpha_model = SpacSignals()
        self.risk_control = TraillingStopLoss(20)

Test = BackTest(SpacInsiderLogisticRegressionModel)
Test.run()

[*********************100%***********************]  70 of 70 completed


0.006943175014047314
0.00461451042038434


In [10]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from server.lib.BackTest import BackTest
from server.lib.models import BaseModel
from server.prebuilt.portfolioAllocations import PercentagePortfolioAllocation
from server.prebuilt.riskManagement import TraillingStopLoss
from server.custom.equitiesSelection import SpacsSelectionModel
from server.custom.strategies import SpacSignals

class SpacInsiderLogisticRegressionModel(BaseModel):
    def __init__(self):
        super().__init__()
        self.start_date = "2022-08-07"
        self.end_date = "2024-08-07"
        self.model_name = 'SPAC Insider Logistic Regression with Trailling Stop Loss 10 Model'
        self.equity_selection_management = SpacsSelectionModel(40, 5, 'Short', self.start_date, self.end_date)
        self.portfolio_management = PercentagePortfolioAllocation(1000000, 10)
        self.alpha_model = SpacSignals()
        self.risk_control = TraillingStopLoss(10)

Test = BackTest(SpacInsiderLogisticRegressionModel)
Test.run()

[*********************100%***********************]  70 of 70 completed


0.004870736158417341
0.0026952027738360914


In [11]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from server.lib.BackTest import BackTest
from server.lib.models import BaseModel
from server.prebuilt.portfolioAllocations import PercentagePortfolioAllocation
from server.prebuilt.riskManagement import TraillingStopLoss
from server.prebuilt.hedgingSystem import BaseHedgeFeature
from server.custom.equitiesSelection import SpacsSelectionModel
from server.custom.strategies import SpacSignals

class SpacInsiderLogisticRegressionModel(BaseModel):
    def __init__(self):
        super().__init__()
        self.start_date = "2022-08-07"
        self.end_date = "2024-08-07"
        self.model_name = 'SPAC Insider Logistic Regression with Trailling Stop Loss 20 with Hedging Model'
        self.equity_selection_management = SpacsSelectionModel(40, 5, 'Short', self.start_date, self.end_date)
        self.portfolio_management = PercentagePortfolioAllocation(1000000, 10)
        self.alpha_model = SpacSignals()
        self.risk_control = TraillingStopLoss(20)
        self.hedging_system = BaseHedgeFeature('QQQ')

Test = BackTest(SpacInsiderLogisticRegressionModel)
Test.run()

[*********************100%***********************]  70 of 70 completed


0.0044263964511997
0.003001310229830933


In [12]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from server.lib.BackTest import BackTest
from server.lib.models import BaseModel
from server.prebuilt.portfolioAllocations import PercentagePortfolioAllocation
from server.prebuilt.riskManagement import TraillingStopLoss
from server.prebuilt.hedgingSystem import BaseHedgeFeature
from server.custom.equitiesSelection import SpacsSelectionModel
from server.custom.strategies import SpacSignals

class SpacInsiderLogisticRegressionModel(BaseModel):
    def __init__(self):
        super().__init__()
        self.start_date = "2022-08-07"
        self.end_date = "2024-08-07"
        self.model_name = 'SPAC Insider Logistic Regression with Trailling Stop Loss 10 with Hedging Model'
        self.equity_selection_management = SpacsSelectionModel(40, 5, 'Short', self.start_date, self.end_date)
        self.portfolio_management = PercentagePortfolioAllocation(1000000, 10)
        self.alpha_model = SpacSignals()
        self.risk_control = TraillingStopLoss(10)
        self.hedging_system = BaseHedgeFeature('QQQ')

Test = BackTest(SpacInsiderLogisticRegressionModel)
Test.run()

[*********************100%***********************]  70 of 70 completed


0.0028947201284282883
0.0016274554473900324
